<a href="https://colab.research.google.com/github/ajaysingh-codes/farmworker-health-rag/blob/main/farmworker_health_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Farmworker Health Research RAG System

**A smart search system for scientific literature on farmworker health, chemical exposures, and occupational stressors**

Built with BM25 + Semantic Search | Interactive comparison interface | Optimized for health research papers

---



## 📦 Installing Required Libraries
This cell installs all the necessary Python packages for our RAG system:
- `sentence-transformers`: For creating semantic embeddings of text
- `bm25s`: For keyword-based search (BM25 algorithm)
- `pypdf2`: For extracting text from PDF files
- `pandas`: For data manipulation
- `numpy`: For numerical operations
- `joblib`: For saving/loading embeddings
- `ipywidgets`: For creating the interactive interface

In [ ]:
!pip install sentence-transformers bm25s pypdf2 pandas numpy joblib ipywidgets -q
print("Libraries installed successfully")

## Importing Libraries and Creating Workspace
This cell:
1. Imports all the libraries we'll use throughout the project
2. Creates a dedicated folder `/content/papers` for your PDF files
3. Sets up the basic environment for our RAG system

In [ ]:
# Import all necessary libraries
import os
import pandas as pd
import numpy as np
import bm25s
import joblib
from sentence_transformers import SentenceTransformer
from IPython.display import display, Markdown
import ipywidgets as widgets
from PyPDF2 import PdfReader
from datetime import datetime

# Create a folder for your PDFs
pdf_folder = "/content/papers"
if not os.path.exists(pdf_folder):
    os.makedirs(pdf_folder)
    print(f"✅ Created folder: {pdf_folder}")
else:
    print(f"📁 Folder already exists: {pdf_folder}")

## Upload Your Research Papers
Upload your 5 PDF papers about farmworker health:
- Click 'Choose Files' to select your PDFs
- Papers will be moved to the `papers` folder
- You'll see confirmation for each uploaded file

In [ ]:
from google.colab import files

print("Click 'Choose Files' below to upload your 5 PDFs:\n")

uploaded = files.upload()

# Move uploaded files to the papers folder
for filename in uploaded.keys():
    destination = os.path.join(pdf_folder, filename)
    os.rename(filename, destination)
    print(f"✅ Uploaded: {filename}")

# Verify and list all PDFs in the folder
pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]
print(f"📊 Total PDFs in folder: {len(pdf_files)}")
for i, pdf in enumerate(pdf_files, 1):
    print(f"  {i}. {pdf}")

## Chunking Function with Overlap
Using 150-word chunks with 20% overlap for optimal context

In [ ]:
from typing import List
import re

def get_chunks_fixed_size_with_overlap(text: str, chunk_size: int, overlap_fraction: float) -> List[str]:
  """
  Splits text into fixed-size chunks with overlap.

  Parameters:
  - text (str): The text to be split into chunks.
  - chunk_size (int): The desired size of each chunk.
  - overlap_fraction (float): The fraction of overlap between chunks (0.2 = 20% overlap)

  Returns:
  - List[str]: A list of text chunks where each chunk might overlap with its adjacent chunk.

  """
  # Split text into individual words
  text_words = text.split()

  # Calculate the number of words to overlap
  overlap_int = int(chunk_size * overlap_fraction)

  #Initialize a list to store resulting chunks
  chunks = []

  # Create chunks with overlap
  for i in range(0, len(text_words), chunk_size):
    # Include overlap from previous chunk
    chunk_words = text_words[max(i - overlap_int, 0): i + chunk_size]

    # Join words to form chunk
    chunk = " ".join(chunk_words)

    chunks.append(chunk)

  return chunks

## 🔍 Extract and Process PDFs
Extract text and create chunks from each paper

In [ ]:
def extract_and_chunk_pdf(pdf_path, chunk_size=150, overlap=0.2):
    """Extract text from PDF and return chunks"""
    try:
        reader = PdfReader(pdf_path)
        full_text = ""

        # Extract all pages
        for page in reader.pages:
            text = page.extract_text()
            if text:
                full_text += text + " "

        # Clean text
        full_text = re.sub(r'\s+', ' ', full_text)
        full_text = re.sub(r'\n+', ' ', full_text)

        # Create chunks
        chunks = get_chunks_fixed_size_with_overlap(full_text, chunk_size, overlap)

        return chunks, len(reader.pages), True

    except Exception as e:
        print(f"❌ Error: {e}")
        return [], 0, False

# Process all PDFs and create dataset
PAPERS_DATA = []

for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_folder, pdf_file)
    paper_name = pdf_file.replace('.pdf', '')

    # Extract and chunk
    chunks, num_pages, success = extract_and_chunk_pdf(pdf_path)

    if success:
        for i, chunk in enumerate(chunks):

            PAPERS_DATA.append({
                'content': chunk, # Main text for search
                'source': paper_name, # Which paper it's from
                'id': f"{paper_name}_{i}" # Pos in paper
            })
    else:
        print(f"  ❌ Failed to process")

print(f"\n✅ Total chunks created: {len(PAPERS_DATA)}")

## Step 3: Implement Retrieval functions
Build BM25 (keyword) and Semantic Search capabilities.

In [ ]:
# Create corpus for searching (combining all text for each chunk)
corpus = [x['content'] for x in PAPERS_DATA]

print(f"Corpus ready with {len(corpus)} chunks")
print(f"From {len(set([x['source'] for x in PAPERS_DATA]))} papers")

### Step 3.1: BM25 (Keyword) Search


In [ ]:
# Create BM25 Retriever
BM25_RETRIEVER = bm25s.BM25(corpus=corpus)

# Tokenize the corpus
TOKENIZED_DATA = bm25s.tokenize(corpus)

# Index the tokenized data
BM25_RETRIEVER.index(TOKENIZED_DATA)

In [ ]:
def bm25_retrieve(query: str, top_k: int = 5):
  """
  BM25 retrieval - keyword-based search
  """

  tokenized_query = bm25s.tokenize(query)

  # Retrieve documents
  results, scores = BM25_RETRIEVER.retrieve(tokenized_query, k=top_k)

  # Get indices
  results = results[0]
  top_k_indices = [corpus.index(result) for result in results]

  return top_k_indices

# Test BM25
test_query = "pesticide exposure health effects"
bm25_results = bm25_retrieve(test_query, top_k=3)

print(f"Query: {test_query}")
for idx in bm25_results[:3]:
  print(f"\n Source: {PAPERS_DATA[idx]['source']}")
  print(f"Preview: {PAPERS_DATA[idx]['content'][:200]}...")

### Step 3.2 : Semantic Search With Embeddings
Create embeddings for semantic understanding

In [ ]:
from sentence_transformers import SentenceTransformer

# Initialize the embedding model
# Using a model suitable for scientific/health content
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Generate embeddings for all chunks
EMBEDDINGS = model.encode(corpus, show_progress_bar=True)

In [ ]:
# Semantic Search Function
def semantic_search_retrieve(query, top_k=5):
  """
  Semantic search using embeddings
  """
  query_embedding = model.encode(query)

  # Calculate cosine similarity
  from sklearn.metrics.pairwise import cosine_similarity
  similarities = cosine_similarity([query_embedding], EMBEDDINGS)[0]

  # Get top-k indices
  top_k_indices = np.argsort(similarities)[-top_k:][::-1]

  return top_k_indices.tolist()

# Test semantic search
semantic_results = semantic_search_retrieve(test_query, top_k=3)

print(f"Query: {test_query}")
for idx in semantic_results[:3]:
  print(f"\n Source: {PAPERS_DATA[idx]['source']}")
  print(f"Preview: {PAPERS_DATA[idx]['content'][:200]}...")

### Step 3.3: Reciprocal Rank Fusion (RRF)
Combine BM25 and Semantic Search using RRF - to reward documents/papers that rank higher in each retrieval technique list.

In [ ]:
# RRF Example
# RRF function (exactly from the example)
def reciprocal_rank_fusion(list1, list2, top_k=5, K=60):
    """
    Combine results from BM25 and Semantic Search
    """
    rrf_scores = {}

    # Calculate RRF scores
    for lst in [list1, list2]:
        for rank, item in enumerate(lst, start=1):
            if item not in rrf_scores:
                rrf_scores[item] = 0
            rrf_scores[item] += 1 / (rank + K)

    # Sort by score
    sorted_items = sorted(rrf_scores, key=rrf_scores.get, reverse=True)

    return sorted_items[:top_k]

# Test RRF
bm25_list = bm25_retrieve(test_query, top_k=5)
semantic_list = semantic_search_retrieve(test_query, top_k=5)
rrf_list = reciprocal_rank_fusion(bm25_list, semantic_list, top_k=5)

print("🔀 Reciprocal Rank Fusion Results:")
print(f"BM25 returned: {bm25_list}")
print(f"Semantic returned: {semantic_list}")
print(f"RRF combined: {rrf_list}")

In [ ]:
def query_papers(indices):
  """
  Retrieve paper chunks by indices
  """
  return [PAPERS_DATA[index] for index in indices]

retrieved_papers = query_papers(rrf_list[:3])
for i, paper in enumerate(retrieved_papers, 1):
  print(f"\n{i}. Source: {paper['source']}")
  print(f"  Content: {paper['content'][:200]}...")